In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
os.kill(os.getpid(), 9)

In [2]:
!pip install -U bitsandbytes

In [ ]:
import torch
from PIL import Image
from transformers import AutoProcessor, AutoModelForVision2Seq, BitsAndBytesConfig
from transformers.image_utils import load_image
import os
import json
from tqdm import tqdm
import pickle

# Specify device
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# Define paths directly
images_path = "drive/MyDrive/smolvlm/archive/food_images/food_images"
pickle_path = "drive/MyDrive/smolvlm/test_titles.pkl"

class Inference:
    def __init__(self, images_path, pickle_path):
        self.images_path = images_path
        self.pickle_path = pickle_path
        # Load the pickle file
        with open(pickle_path, 'rb') as f:
            # A dictionary with keys 'image_name' and 'title'
            self.titles = pickle.load(f)

        self.processor = AutoProcessor.from_pretrained("HuggingFaceTB/SmolVLM-Base")
        quantization_config = BitsAndBytesConfig(load_in_8bit=True)
        self.model = AutoModelForVision2Seq.from_pretrained(
            "HuggingFaceTB/SmolVLM-Instruct",
            quantization_config=quantization_config,
        )


    def load_image(self, image_path):
        image = load_image(image_path)
        return image

    def generate_description(self, image_path):
        image = self.load_image(image_path)
        messages = [
            {
                "role": "user",
                "content": [
                    {"type": "image"},
                    {"type": "text", "text": "Can you give a title to this dish?"}
                ]
            },
        ]
        prompt = self.processor.apply_chat_template(messages, add_generation_prompt=True)
        inputs = self.processor(text=prompt, images=[image], return_tensors="pt")
        inputs = inputs.to(DEVICE)
        generated_ids = self.model.generate(**inputs, max_new_tokens=50)
        generated_texts = self.processor.batch_decode(
            generated_ids,
            skip_special_tokens=True,
        )
        print(generated_texts[0])
        if os.path.splitext(os.path.basename(image_path))[0] in self.titles:
            ground_truth = self.titles[os.path.splitext(os.path.basename(image_path))[0]]
        print("GT:", ground_truth)
        return (generated_texts[0], f"Ground truth: {ground_truth}", os.path.basename(image_path))

    def save_to_json(self, descriptions, output_file='descriptions.json'):
        with open(output_file, 'w') as f:
            json.dump(descriptions, f)

    def process_images(self, output_file='descriptions.json'):
        descriptions = []
        image_names = os.listdir(self.images_path)
        image_names = [
                      image_name for image_name in image_names
                      if os.path.splitext(os.path.basename(image_name))[0] in self.titles
                      ]

        progress_bar = tqdm(image_names, desc="Processing Images", unit="image")
        for image_name in progress_bar:
            image_path = os.path.join(self.images_path, image_name)
            if os.path.isfile(image_path):
                description = self.generate_description(image_path)
                descriptions.append(description)
        self.save_to_json(descriptions, output_file)

# Instantiate the class
inference = Inference(images_path, pickle_path)

# Process the images and save descriptions
output_file = "descriptions.json"  # Define output file
inference.process_images(output_file)

# Load and view saved descriptions
with open(output_file, 'r') as f:
    descriptions = json.load(f)

# Display some results
print(descriptions[:5])


Some kwargs in processor config are unused and will not have any effect: image_seq_len. 
`low_cpu_mem_usage` was None, now default to True since model is quantized.
Processing Images:   0%|          | 1/1252 [00:03<1:12:27,  3.48s/image]

User:<image>Can you give a title to this dish?
Assistant: Strawberry Rhubarb Cream Cheese Spread.
GT: Toasted Nori Mayonnaise


Processing Images:   0%|          | 2/1252 [00:06<1:04:22,  3.09s/image]

User:<image>Can you give a title to this dish?
Assistant: Creamy White Bean Soup.
GT: Tomato and Cannellini Bean Soup


Processing Images:   0%|          | 3/1252 [00:09<1:05:45,  3.16s/image]

...

User:<image>Can you give a title to this dish?
Assistant: Crema catalana.
- Facts:
- The image displays a large, round, caramel-colored dessert on a white plate.
- The dessert is sitting in a pool of dark brown liquid.
- The plate is placed on a
GT: Coffee Flan


Processing Images:  80%|███████▉  | 998/1252 [1:25:09<24:28,  5.78s/image]

User:<image>Can you give a title to this dish?
Assistant: Chicken and Potatoes in a Red Pot.
GT: Cod en Cocotte with Tomatoes, Olives & Chorizo


Processing Images:  80%|███████▉  | 999/1252 [1:25:12<21:15,  5.04s/image]

User:<image>Can you give a title to this dish?
Assistant: Gingerbread cake.
GT: Coffee-Spice Shortbread with Crystallized Ginger
